In [1]:
import pandas as pd

In [2]:
uber = pd.read_csv("datasets/uber.csv")
uber.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [3]:
uber.shape

(200000, 9)

In [4]:
uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [5]:
uber=uber.drop("Unnamed: 0", axis=1)
uber.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [6]:
uber['dropoff_longitude'].dropna()
uber['dropoff_latitude'].dropna()
uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key                200000 non-null  object 
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 12.2+ MB


In [34]:
'''
We need to clean our latitude and longitude values
 This is because there are legitimate bounds to lat and long
 the limits are as such : latitude[-90,90], latitude[-180,180]
However, there is a bug with the input of coords so we could
 only make it work with -90 to 90 for both
'''

uber_filtered = uber[uber['pickup_longitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['pickup_longitude'] > -90]
print(f"{uber_filtered['pickup_longitude'].max()}, {uber_filtered['pickup_longitude'].min()}")

uber_filtered = uber_filtered[uber_filtered['pickup_latitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['pickup_latitude'] > -90]
print(f"{uber_filtered['pickup_latitude'].max()}, {uber_filtered['pickup_latitude'].min()}")

uber_filtered = uber_filtered[uber_filtered['dropoff_longitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['dropoff_longitude'] > -90]
print(f"{uber_filtered['dropoff_longitude'].max()}, {uber_filtered['dropoff_longitude'].min()}")

uber_filtered = uber_filtered[uber_filtered['dropoff_latitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['dropoff_latitude'] > -90]
print(f"{uber_filtered['dropoff_latitude'].max()}, {uber_filtered['dropoff_latitude'].min()}")


40.808425, -89.93333299999999
48.01876, -74.01551500000001
40.831932, -75.458978633981
45.031598, -74.01575


40.808425
48.01876
----------------
-93.824668
-74.01551500000001
----------------
40.831932
45.031598
----------------
-75.458978633981
-74.01575


In [21]:
uber.shape

(199988, 8)

In [28]:
uber.iloc[[40906]]

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
40908,2009-07-30 19:51:48.0000005,8.1,2009-07-30 19:51:48 UTC,-93.824668,43.486656,-54.294363,35.174169,1


In [36]:
import geopy.distance
def transform_coords(df):
    result_list = []
    counter = 0

    for index, row in df.iterrows():
        coords_1 = (row['pickup_longitude'], row['pickup_latitude'])
        coords_2 = (row['dropoff_longitude'], row['dropoff_latitude'])

        # print(f"i: {index} : {geopy.distance.geodesic(coords_1, coords_2).miles}")
        result_list.append(geopy.distance.geodesic(coords_2, coords_1).miles)
    return result_list

distances = transform_coords(uber_filtered)

uber_filtered['distance_in_miles'] = distances
uber_filtered.head(10)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_in_miles
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,0.290272
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,0.423525
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,3.008397
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,0.786895
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,3.349106
5,2011-02-12 02:27:09.0000006,4.9,2011-02-12 02:27:09 UTC,-73.969019,40.755910,-73.969019,40.755910,1,0.000000
6,2014-10-12 07:04:00.0000002,24.5,2014-10-12 07:04:00 UTC,-73.961447,40.693965,-73.871195,40.774297,5,6.446615
7,2012-12-11 13:52:00.00000029,2.5,2012-12-11 13:52:00 UTC,0.000000,0.000000,0.000000,0.000000,1,0.000000
8,2012-02-17 09:32:00.00000043,9.7,2012-02-17 09:32:00 UTC,-73.975187,40.745767,-74.002720,40.743537,1,1.909893
9,2012-03-29 19:06:00.000000273,12.5,2012-03-29 19:06:00 UTC,-74.001065,40.741787,-73.963040,40.775012,1,2.712677


In [37]:
uber_filtered['distance_in_miles']

0         0.290272
1         0.423525
2         3.008397
3         0.786895
4         3.349106
            ...   
199995    0.040030
199996    1.523166
199997    8.896701
199998    1.122834
199999    0.935749
Name: distance_in_miles, Length: 199986, dtype: float64